# Skincancer HAM-dataset med Pytorch/ONNX from Matlab

### Standardimporter

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision
from torchvision import datasets, transforms, models
from torchvision.utils import make_grid
import torch.optim as optim
from torch.optim import lr_scheduler
import os


import numpy as np
import pandas as pd
#from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
import onnx

In [24]:
do_training = True

### Ladda en modell:

In [12]:
!pwd

/home/johan/coding/python-ml/pytorch/skincancer_ham_pytorch


In [13]:
# Nedanstående två rader måste göras oavsett träning eller ej:

# Load the ONNX model
onnx_filename = "trained_models/alexNet_bal_data.onnx"
model = onnx.load(onnx_filename)

In [28]:
# Check that the IR is well formed
onnx.checker.check_model(model)

# Print a human readable representation of the graph
# onnx.helper.printable_graph(model.graph)

# print(model)

### Definiera de olika mängderna för träning, validering och test:

In [18]:
batchsz = 32
train_num_workers = 4
test_val_num_workers = 8

# Tror matlab-modellen är tränad för 227x227...:
img_w = 227
img_h = 227
img_dim = 3


# Definiera vilken augmentation som ska göras:
train_data_transform = transforms.Compose([
        transforms.RandomResizedCrop(img_w),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                             std=[0.229, 0.224, 0.225])
    ])

val_test_data_transform = transforms.Compose([
        transforms.RandomResizedCrop(img_w),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                             std=[0.229, 0.224, 0.225])
    ])

basepath = "../../../ml/Datasets/skin-cancer-mnist-ham10000/images_per_label_splitted_sets/"

# Definiera de tre dataseten:
skincancer_train_dataset = datasets.ImageFolder(root = basepath + 'train/',
                                           transform = train_data_transform)

skincancer_valid_dataset = datasets.ImageFolder(root = basepath + 'val/',
                                           transform = val_test_data_transform)

skincancer_test_dataset = datasets.ImageFolder(root = basepath + 'test/',
                                              transform = val_test_data_transform)

In [19]:
batchsz_val = 2**5
batchsz_test = 2**5

# Skapa loaders för de tre dataseten:
train_loader = torch.utils.data.DataLoader(skincancer_train_dataset,
                                             batch_size = batchsz, 
                                             shuffle = True,
                                             pin_memory = True,
                                             drop_last = True,
                                             num_workers = train_num_workers)

valid_loader = torch.utils.data.DataLoader(skincancer_valid_dataset,
                                             batch_size = batchsz, 
                                             shuffle = True,
                                             num_workers = test_val_num_workers)

test_loader = torch.utils.data.DataLoader(skincancer_test_dataset,
                                             batch_size = batchsz_test, 
                                             shuffle = False,
                                             num_workers = test_val_num_workers)

In [22]:
trainableparameters = []
for param in model.parameters():
    # trainableparameters = param.numel()
    trainableparameters.append(param.numel())

print("Antalet träningsbara parametrar är =", np.sum(trainableparameters), "st.")


AttributeError: parameters

### Definiera loss-function och vilken metod för optimering som ska användas:

In [21]:
# Definiera loss-function och vilken optimerare som ska användas:

learning_rate = 1e-4
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

AttributeError: parameters

### Utvärdera på valideringsmängden:

In [26]:
# Sätt modellen i utvärderingsläge:
#model.eval()

valid_correct_pred_per_epoch = 0
valid_accuracy = []
valid_losses = []

no_valid_images = len(valid_loader.dataset) # = 998
with torch.no_grad():
    # Antal iterationer = 998 / 16 = 63 st.
    for X_valid, y_valid in valid_loader:

        y_valid_pred = model(X_valid)
        
        valid_predicted = torch.max(y_valid_pred.data, 1)[1]
        valid_is_correct = (valid_predicted == y_valid).sum()
        valid_correct_pred_per_epoch += valid_is_correct
        
        valid_loss = criterion(y_valid_pred, y_valid)
        valid_losses.append(valid_loss.item())
            
        valid_epoch_accuracy = valid_correct_pred_per_epoch.item() / no_valid_images
        valid_accuracy.append(valid_epoch_accuracy)
        
# epoch startar på 0, därav "+1" nedan:
print(f"Validering: accuracy = {valid_epoch_accuracy:4.4f} och loss är = {valid_loss:4.4f}")

TypeError: 'ModelProto' object is not callable

### Utvärdera modellen på test-set

In [89]:
import time

model.eval()

no_test_images = len(valid_loader.dataset) #len(test_loader.dataset)

start_eval_test_time = time.time()
with torch.no_grad():
    correct = 0
    # Antal iterationer = 2010 / 16 = 123 st.
    for X_test, y_test in valid_dl:
        y_pred_test = model.forward(X_test)
        predicted = torch.max(input = y_pred_test, dim = 1)[1]
        correct += (predicted == y_test).sum()
    end_eval_test_time = time.time()
eval_test_time = end_eval_test_time - start_eval_test_time
print(f'Test accuracy: {correct.item()}/{no_test_images} = {correct.item()*100/(no_test_images):5.2f} %')
print("\nEvaluation took", eval_test_time, "seconds.")

OSError: Caught OSError in DataLoader worker process 5.
Original Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\envs\pytorch_general\lib\site-packages\torch\utils\data\_utils\worker.py", line 178, in _worker_loop
    data = fetcher.fetch(index)
  File "C:\ProgramData\Anaconda3\envs\pytorch_general\lib\site-packages\torch\utils\data\_utils\fetch.py", line 44, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "C:\ProgramData\Anaconda3\envs\pytorch_general\lib\site-packages\torch\utils\data\_utils\fetch.py", line 44, in <listcomp>
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "C:\ProgramData\Anaconda3\envs\pytorch_general\lib\site-packages\torchvision\datasets\folder.py", line 138, in __getitem__
    sample = self.loader(path)
  File "C:\ProgramData\Anaconda3\envs\pytorch_general\lib\site-packages\torchvision\datasets\folder.py", line 174, in default_loader
    return pil_loader(path)
  File "C:\ProgramData\Anac

### Confusion matrix

In [73]:
from sklearn.metrics import confusion_matrix
number_of_classes = len(test_loader.dataset.classes)

In [74]:
y_true = test_loader.dataset.targets

In [48]:
X_test = test_loader.

In [ ]:
confusion_matrix(y_true, y_pred)

In [43]:
def pytorch_confusion_matrix(num_classes, model, dataloader):
    confusion_matrix = torch.zeros(num_classes, num_classes)
    with torch.no_grad():
        for i, (inputs, classes) in enumerate(test_loader):
            inputs = inputs.to(device)
            classes = classes.to(device)
            predictions = model(inputs)
            _, preds = torch.max(predictions, 1)
            for t, p in zip(classes.view(-1), preds.view(-1)):
                confusion_matrix[t.long(), p.long()] += 1
    return print(confusion_matrix)

In [44]:
cm_test = pytorch_confusion_matrix(number_of_classes, model, test_loader)

tensor([[567., 158., 201.,  52.,  70.,  17.,   7.],
        [ 69., 974.,  75.,  23.,  57.,  30.,  20.],
        [ 33.,  24., 657.,  17.,  93.,  55.,   5.],
        [ 24.,  72.,  53., 501.,  58.,  37.,  23.],
        [  3.,  23., 182.,   4., 572.,  92.,  16.],
        [  5.,  31., 121.,  22., 179., 967.,  17.],
        [  1.,  15.,  11.,  27.,  24.,  16., 834.]])


In [45]:
cm_train = pytorch_confusion_matrix(number_of_classes, model, train_loader)

tensor([[545., 182., 188.,  52.,  83.,  19.,   3.],
        [ 69., 965.,  72.,  25.,  64.,  21.,  32.],
        [ 28.,  26., 643.,  20., 112.,  49.,   6.],
        [ 24.,  61.,  60., 517.,  58.,  42.,   6.],
        [ 14.,  24., 184.,   7., 555.,  93.,  15.],
        [ 10.,  31., 101.,  15., 190., 971.,  24.],
        [  3.,  13.,  10.,  18.,  30.,   9., 845.]])


In [ ]:
cm_train_np = np.asarray(cm_train)

In [ ]:
cm_train_np.shape